In [15]:
a=['a','b','b','c']
b=['w','x','y','z']
print(a)
print(b)
#for letter in a:
#    if (letter=='c'):
#       print(a.index(letter))

for index in range(len(a)):
    if a[index]=='b':
        del a[index]
        del b[index]
        
print(a)
print(b)


['a', 'b', 'b', 'c']
['w', 'x', 'y', 'z']


IndexError: list index out of range

In [21]:
a=['a','b','b','c']
b=['w','x','y','z']
c=[entry for entry in a if entry !='a']
print(a)
print(b)
print(c)
for letter in a:
    if (letter=='c'):
        del letter
print(a)
print(b)

['a', 'b', 'b', 'c']
['w', 'x', 'y', 'z']
['b', 'b', 'c']
['a', 'b', 'b', 'c']
['w', 'x', 'y', 'z']


In [28]:
c=[[123,['a','b']],[456,['e','f']],[789,['e','f']],[788,['e','f']]]
['a','b'] in c
d=[entry for entry in c if entry[0] > 789-60]
print(d)

[[789, ['e', 'f']], [788, ['e', 'f']]]


In [47]:
from itertools import combinations
listOfHashtags=['ht2','ht1','ht3']
new_list_of_edges=list(combinations(listOfHashtags,2))
print (new_list_of_edges)
for entry in new_list_of_edges:
    print(entry)
    listEntry=list(entry)
    print(listEntry.sort())
print (new_list_of_edges)
#this works
output=[sorted(entry) for entry in new_list_of_edges]
print(output)

[('ht2', 'ht1'), ('ht2', 'ht3'), ('ht1', 'ht3')]
('ht2', 'ht1')
None
('ht2', 'ht3')
None
('ht1', 'ht3')
None
[('ht2', 'ht1'), ('ht2', 'ht3'), ('ht1', 'ht3')]
[['ht1', 'ht2'], ['ht2', 'ht3'], ['ht1', 'ht3']]


In [49]:
def create_edge_entries(listOfHashtags):
    new_list_of_tuple_edges=list(combinations(listOfHashtags,2))
    new_list_of_edges = [sorted(element) for element in new_list_of_tuple_edges]
    return new_list_of_edges

x=['xx','ab','ac','aa']
create_edge_entries(x)

[['ab', 'xx'],
 ['ac', 'xx'],
 ['aa', 'xx'],
 ['ab', 'ac'],
 ['aa', 'ab'],
 ['aa', 'ac']]

In [50]:
import sys
import os
import json #use for json parser
from itertools import combinations #used to run combinations (order doesn't matter)
import time #needed to deal with timestamps

#Note: From the way run.sh is called:
#print(sys.argv[0]) #will be the name of this script: average_degree.py
#print(sys.argv[1]) #will be ./tweet_input/tweets.txt 
#print(sys.argv[2]) #Will be ./tweet_output/output.txt

#todo replace:
# sys_argv0 replace for sys.argv[0]
# sys_argv1
# sys_argv2
sys_argv1 = './tweet_input/tweets.txt'
sys_argv2 = './tweet_output/output.txt'
#todo:
#remove tweet_input/tweets.txt from the gitignore

#1.     Get tweets:
#1a     If it's a rate limiting message, ignore it

#2.     Check timestamp:
#2a     If timestamp is older than 60s, delete tweet, jump to call calc_average_degree() to end
#2b     If timestamp is newer than newest, update newest_timestamp value

#3      Delete edges that are older than 60 seconds

#4      Find hashtags:
#4a     If tweet has 2 or most hashtags, check and remove all duplicates
#4b     If only 0 or 1 hashtag remains, discard tweet, jump to call calc_average_degree()

#5      Create edge entries: (If tweet has 2 or more valid hashtags, create edge entries)
#5a     Use the combination package that was imported. Eg: list(combinations(['hashtag1','hashtag2','hashtag3'],2)). This outputs a list of tuples.
#5b     Sort each edge entry alphabetically so that we don't have the check the reverse. Do this by converting each tuple into a list and sorting

#6      Insert each new edge entry into edge_list:
#6a     Check that the edge (and the reverse) doesn't already exist, if it does, remove the older edge

#7      call calc_average_degree()
#7a     concatenate the 2 columns of nodes in the edge_list, and sum
#7b     remove duplicates to get the number of nodes



#Implementation of the above outline:

NEWEST_TIMESTAMP=0.00; #global vairable
EDGE_LIST=[] #of the format: [[timestamp1,['hashtagX1','hashtagY1']], [timestamp2,['hashtagX2','hashtagY2']], ...etc]

#clear out old ./tweet_output/tweets.txt file
if os.path.exists(sys_argv2):
    os.remove(sys_argv2) #note that the location is based on where run.sh was called

outputFile = open(sys_argv2, 'a+')

#1. Get tweets

#future optimization: this may be able to be optimized to process one-at-a-time as each json object line in the text file is read
tweets = []
for line in open(sys_argv1, 'r'):
    line_json_parsed = json.loads(line)
    if 'created_at' in line_json_parsed: #1a. Ignore the rate limiting messages
        tweets.append(line_json_parsed)
        # #Example usage of the tweets array that examples the 248th tweet in the file
        # tweets[248] # fetches a tweet
        # tweets[248]['created_at'] # fetches timestamp of tweet, returns in the format of "Wed Aug 29 17:12:58 +0000 2012"
        # tweets[248]['entities']['hashtags'] # fetches all hashtags of a tweet
        # tweets[248]['entities']['hashtags'][1] # fetches individual hashtag of a tweet
        # tweets[248]['entities']['hashtags'][1]['text'] # fetch individual hashtag text of a tweet
        # len(tweets[248]['entities']['hashtags']) #tells you how many hashtags the tweet had

for tweet in tweets:
    tweet_timestamp = time.mktime(time.strptime(tweet['created_at'],"%a %b %d %H:%M:%S +0000 %Y")) #followed this http://stackoverflow.com/questions/18604755/twitter-created-at-convert-epoch-time-in-python
    if (check_and_update_timestamp(tweet_timestamp)):     #2. Check timestamp: #note that this created_at field is always utc time
        global EDGE_LIST = [entry for entry in EDGE_LIST if entry[0] > NEWEST_TIMESTAMP-60] #3 Delete edges older than 60s
        validHashtags=find_hashtags(tweet)         #4      Find hashtags:
        if(len(validHashtags)>1):
            new_list_of_edges=create_edge_entries(validHashtags);             #5      Create edge entries:
            update_edge_list(new_list_of_edges,tweet_timestamp);
    calc_average_degree();             #6      Insert each new edge entry into edge_list:

outputFile.close();







####### Functions are defined below #######

#2.     Check timestamp:
#2a     If timestamp is older than 60s, delete tweet, jump to call calc_average_degree()
#2b     If timestamp is newer than newest, update newest_timestamp value
def check_and_update_timestamp( timestampToBeChecked_epoch_utc ): 
     if timestampToBeChecked_epoch_utc < global NEWEST_TIMESTAMP-60:
        return false
    else if timestampToBeChecked_epoch_utc > global NEWEST_TIMESTAMP:
        global NEWEST_TIMESTAMP=timestampToBeChecked_epoch_utc;
    return true #whether or not this was the newest timestamp or just a valid one, return true
 
#4      Find Hashtags:NEWEST_TIMESTAMP;
#4a     If tweet has 2 or most hashtags, check and remove all duplicates
#4b     If only 0 or 1 hashtag remains, discard tweet
def find_hashtags(tweet):
    hashtags=[]
    if (len(tweet['entities']['hashtags'])>0):
        for tweet_entity_hashtag in tweet['entities']['hashtags']:
            hashtags.append(tweet_entity_hashtag['text']);
        hashtags=list(set(hashtags)) #remove duplicates. Note that this could still return just 1 hashtag
    return hashtags

#5a     Use the combination package that was imported. Eg: list(combinations(['hashtag1','hashtag2','hashtag3'],2)). This outputs a list of tuples.
#5b     Sort each edge entry alphabetically so that we don't have the check the reverse. Do this by converting each tuple into a list and sorting
def create_edge_entries(listOfHashtags):
    new_list_of_tuple_edges=list(combinations(listOfHashtags,2))
    new_list_of_edges = [sorted(element) for element in new_list_of_tuple_edges]
    return new_list_of_edges

#6      Insert each new edge entry into edge_list:
#6a     Check that the edge doesn't already exist, if it does, update timestamp of that edge (no need to check for reverse order, because each edge entry is already sorted)
def update_edge_list(new_list_of_edges,tweet_timestamp):
    for new_edge_to_be_added in new_list_of_edges
        for element in EDGE_LIST:
            if element[1]==new_edge_to_be_added:
                element[0]= tweet_timestamp;
                break;
        #looped through whole existing EDGE_LIST and didn't break out of 2nd for loop, meaning this is a new edge
        EDGE_LIST.append([tweet_timestamp,new_edge_to_be_added])
    return true;

#7      call calc_average_degree()
#7a     concatenate the 2 columns of nodes in the edge_list, and sum
#7b     remove duplicates to get the number of nodes
def calc_average_degree():
    all_edges=getColumn(EDGE_LIST,1)
    sumDegrees=2*len(all_edges);
    list_of_all_nodes = set([item for sublist in all_edges for item in sublist]) #followed this: http://stackoverflow.com/questions/952914/making-a-flat-list-out-of-list-of-lists-in-python
    num_nodes=len(list_of_all_nodes)
    average_degree_untruncated=0.00
    average_degree_untruncated=sumDegrees/num_nodes;
    average_degree=float('%.2f'%(average_degree_untruncated));
    outputFile.write(average_degree)
    return true;

#helper function for getting columns
def getColumn(matrix, i):
    return [row[i] for row in matrix]

SyntaxError: invalid syntax (<ipython-input-50-6a217dd1967a>, line 77)